In [1]:
from split_buck import split_buck
from mincut_pool_net import Net
from TrainSet import TrainSet
from MTL import MTLNetwork

import torch
from torch.utils.data import DataLoader
import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset

max_nodes = 111
dataset = TUDataset(root='./tmp', name='NCI1')
torch.manual_seed(12315)
dataset = dataset.shuffle()
# dataset_2 = dataset
dataset_length = len(dataset)

transform = T.ToDense(max_nodes)

D_aux = dataset[int(0.4 * dataset_length): int(0.7 * dataset_length)]
G_target = dataset[int(0.7 * dataset_length):]

buck_num = 8

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = './NCI_model_mincut_pool.pt'
model = Net(dataset.num_features, dataset.num_classes)
model = torch.load(model_path)
model = model.to(device)

D_aux_graph_embedding = []
D_aux_num_nodes_raw = []
D_aux_num_edges_raw = []
D_aux_graph_density_raw = []

for index, data in enumerate(D_aux):
    D_aux_num_nodes_raw.append(data.num_nodes)
    D_aux_num_edges_raw.append(data.num_edges)

    density_temp = 2 * data.num_nodes / (data.num_edges * (data.num_edges - 1))
    D_aux_graph_density_raw.append(density_temp)

    data = transform(data).to(device)
    _, _, _, test_output = model(data.x, data.adj)
    D_aux_graph_embedding.append(test_output.squeeze(dim=0).cpu())

del test_output, data, index, model

In [3]:
D_aux_length = len(D_aux_num_nodes_raw)

D_aux_num_nodes_raw_train = D_aux_num_nodes_raw[:int(0.6 * D_aux_length)]
D_aux_num_nodes_raw_test = D_aux_num_nodes_raw[int(0.6 * D_aux_length):]

D_aux_num_edges_raw_train = D_aux_num_edges_raw[:int(0.6 * D_aux_length)]
D_aux_num_edges_raw_test = D_aux_num_edges_raw[int(0.6 * D_aux_length):]

D_aux_graph_embedding_train = D_aux_graph_embedding[:int(0.6 * D_aux_length)]
D_aux_graph_embedding_test = D_aux_graph_embedding[int(0.6 * D_aux_length):]

D_aux_graph_density_train = D_aux_graph_density_raw[0:int(0.6 * len(D_aux_graph_density_raw))]
D_aux_graph_density_test = D_aux_graph_density_raw[int(0.6 * len(D_aux_graph_density_raw)):]

In [4]:
D_aux_nodes_train = split_buck(D_aux_num_nodes_raw_train, buck_num)
D_aux_nodes_test = split_buck(D_aux_num_nodes_raw_test, buck_num)

D_aux_edges_train = split_buck(D_aux_num_edges_raw_train, buck_num)
D_aux_edges_test = split_buck(D_aux_num_edges_raw_test, buck_num)

D_aux_density_train = split_buck(D_aux_graph_density_train, buck_num)
D_aux_density_test = split_buck(D_aux_graph_density_test, buck_num)

In [5]:
myDataset = TrainSet(D_aux_graph_embedding_train, D_aux_nodes_train, D_aux_edges_train, D_aux_density_train)
train_loader = DataLoader(myDataset, batch_size=200, shuffle=True)

In [6]:
model = MTLNetwork(buck_num).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [7]:
def train():
    model.train()
    total_loss = 0

    for data in train_loader:
        inputs, num_nodes, num_edges, graph_density = data
        optimizer.zero_grad()
        inputs = inputs.detach()
        num_nodes = num_nodes.detach()
        num_edges = num_edges.detach()
        graph_density = graph_density.detach()

        pred_nodes, pred_edges, pred_density = model(inputs.to(device))
        loss_1 = criterion(pred_nodes, num_nodes.type(torch.LongTensor).to(device))
        loss_2 = criterion(pred_edges, num_edges.type(torch.LongTensor).to(device))
        loss_3 = criterion(pred_density, graph_density.type(torch.LongTensor).to(device))

        loss = loss_1 + loss_2 + loss_3

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(D_aux_nodes_train)

In [8]:
myTestData = TrainSet(D_aux_graph_embedding_test, D_aux_nodes_test, D_aux_edges_test, D_aux_density_test)
test_loader = DataLoader(myTestData, batch_size=1, shuffle=False)

In [9]:
@torch.no_grad()
def test():
    model.eval()
    total_loss = 0
    accu_nodes = 0
    accu_edges = 0
    accu_density = 0

    for data in test_loader:
        inputs, num_nodes, num_edges, graph_density = data
        inputs = inputs.detach()
        num_nodes = num_nodes.detach()
        num_edges = num_edges.detach()
        graph_density = graph_density.detach()

        pred_nodes, pred_edges, pred_density = model(inputs.to(device))

        pred_nodes_argmax = torch.argmax(pred_nodes, dim=1).type(torch.float).cpu()
        pred_edges_argmax = torch.argmax(pred_edges, dim=1).type(torch.float).cpu()
        pred_density_argmax = torch.argmax(pred_density, dim=1).type(torch.float).cpu()
        # print(pred_density_argmax, '      ', graph_density)

        if torch.equal(pred_nodes_argmax, num_nodes):
            accu_nodes += 1

        if torch.equal(pred_edges_argmax, num_edges):
            accu_edges += 1

        if torch.equal(pred_density_argmax, graph_density):
            accu_density += 1

        loss_1 = criterion(pred_nodes, num_nodes.type(torch.LongTensor).to(device))

        loss_2 = criterion(pred_edges, num_edges.type(torch.LongTensor).to(device))

        loss_3 = criterion(pred_density, graph_density.type(torch.LongTensor).to(device))

        loss = loss_1 + loss_2 + loss_3

        total_loss += loss.item()

    return total_loss / len(D_aux_nodes_test), accu_nodes / len(D_aux_nodes_test), accu_edges / len(
        D_aux_nodes_test), accu_density / len(D_aux_nodes_test)

In [10]:
test_accu_record = []
for epoch in range(60):
    train_loss = train()
    test_loss, test_nodes_accu, test_edges_accu, test_density_accu = test()
    test_accu_record.append([test_nodes_accu, test_edges_accu, test_density_accu])
    print(
        f'Epoch: {epoch:4d}, Train loss: {train_loss:.4f}, Test loss: {test_loss:.4f}, Test node accu: {test_nodes_accu:.4f}, Test edge accu: {test_edges_accu:.4f}, Test density accu: {test_density_accu:.4f}')

Epoch:    0, Train loss: 0.0338, Test loss: 6.2349, Test node accu: 0.0182, Test edge accu: 0.0628, Test density accu: 0.1235
Epoch:    1, Train loss: 0.0337, Test loss: 6.2264, Test node accu: 0.1761, Test edge accu: 0.1174, Test density accu: 0.1235
Epoch:    2, Train loss: 0.0337, Test loss: 6.2173, Test node accu: 0.1640, Test edge accu: 0.1174, Test density accu: 0.1761
Epoch:    3, Train loss: 0.0336, Test loss: 6.2058, Test node accu: 0.2166, Test edge accu: 0.1174, Test density accu: 0.1883
Epoch:    4, Train loss: 0.0336, Test loss: 6.1913, Test node accu: 0.2510, Test edge accu: 0.1174, Test density accu: 0.2126
Epoch:    5, Train loss: 0.0335, Test loss: 6.1727, Test node accu: 0.2632, Test edge accu: 0.1397, Test density accu: 0.2328
Epoch:    6, Train loss: 0.0334, Test loss: 6.1485, Test node accu: 0.2652, Test edge accu: 0.2024, Test density accu: 0.2409
Epoch:    7, Train loss: 0.0332, Test loss: 6.1174, Test node accu: 0.2672, Test edge accu: 0.2247, Test density accu:

In [11]:
import numpy as np

test_accu_record = np.array(test_accu_record[-5:])
np.mean(test_accu_record, axis=0)

array([0.66801619, 0.67935223, 0.61295547])